# Econometrics for ECO/EBE Homework Grading and Incentives

## Jaap Abbring, 29 January 2018

For many years, students in Econometrics for ECO/EBE had to hand in 11 individual homework assignments, which were graded pass or fail. They also had to hand in a larger group assignment and individually take a final exam, both of which were graded on a 0-10 scale. Over time, we changed the scheme to compute the final grade from the two grades and the number of individual homework assignments passed, to improve the incentives for students to regularly hand in assigments. Below, we solve a student's optimal decision rule for handing in individual assignments, for a given grading scheme. You can change the grading scheme and/or the student's primitives (including exam performance, etcetera) to see how this affects the student's choices to hand in the individual assignments.

### Grading Scheme

First, we set the number of homework assignments `nAss` and specify a function `gradeScheme` that maps the grade for the group assignment (`hwGrade`), the number of individual homework assignments passed (`assPassed`) and the grade for the final exam (`finGrade`) into an overall grade for the course. Two example grade schemes are given, a 'harsh' one that we initially used and a 'smooth' one that we later switched to (one should be commented out). In fact, the final exam counted for 75% of the grade, but one may want to put a higher weight than 25% on the homework grades to increase incentives.

In [ ]:
nAss = 11;

gradeScheme=@(hwGrade,assPassed,finGrade)(0.25*hwGrade*(assPassed>nAss-3)+0.75*finGrade); % Harsh
% gradeScheme=@(hwGrade,assPassed,finGrade)(0.4*hwGrade*assPassed/nAss+0.6*finGrade); % Smooth

### Student Primitives

The student maximizes expected utility, discounted with factor `rho`. In each period `ass`, the student either works, at a utility cost `effortCost(ass)`, or shirks, at a zero utility cost. If the student works, she passes assignment `ass` with probability `successPr`; if she shirks, she fails it for sure. Magically, the grade `hwGrade` for the group assignment is known and independent of effort. The final exam is taken in period `nAss`+1; its grade is drawn, again independently of the student's effort, from a discrete distribution on $\{0,1,2,\ldots,10\}$ encoded in the $1\times 11$ probability vector `finGradePr`. Right after that, the student gets utility `u(grade)` from the overall grade `grade` computed from `hwGrade`, `assPassed` and `finGrade` using `gradeScheme`. 

Here, it is particularly interesting to experiment with the probability `succesPr` taking e.g. the values 1, 0.95, 0.9 and 0.7. 

In [ ]:
effortCost = ones(1,nAss)*0.3;
successPr = 1;
hwGrade = 10;
finGradePr = [0 0.05 0.05 0.1 0.15 0.15 0.2 0.15 0.10 0.05 0];
u=@(grade)(grade-10*(grade<5.25));
rho = 0.9;

In [ ]:
% plot(0:0.01:10,u(0:0.01:10))

### Initialization

In [ ]:
v=nan(nAss+1,nAss+1); UWork=nan(nAss,nAss); UShirk=UWork; work=UWork;
v(nAss+1,:) = finGradePr*u(gradeScheme(hwGrade,ones(11,1)*(0:nAss),(0:10)'*ones(1,nAss+1)));

### Solve by Backward Recursion

In [ ]:
for ass=nAss:-1:1
    UShirk(ass,1:ass) = rho*v(ass+1,1:ass);
    UWork(ass,1:ass) = successPr*rho*v(ass+1,2:ass+1)+(1-successPr)*UShirk(ass,1:ass)-effortCost(ass);
    work(ass,1:ass)=UWork(ass,1:ass)>UShirk(ass,1:ass);
    v(ass,1:ass) = max([UWork(ass,1:ass);UShirk(ass,1:ass)]);
end

In [ ]:
UShirk
UWork
work